In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import warnings
warnings.filterwarnings('ignore')

# Data Cleaning

## Metro Bike Share Data

Metro Bike Share provides the following information regarding the cleanliness of their data:

> Data will be cleansed prior to publication according to the following criteria:
> * Staff servicing and test trips are removed.
> * Trips below 1 minute are removed.
> * A "Virtual Station" listed in the checkout and return kiosks, is used by staff to check in or check out a bike remotely for a special event or in a situation in which a bike could not otherwise be checked in or out to a station.
> * Trip lengths are capped at 24 hours.
> * Some short round trips or long trips may be the result of system or user error, but have been kept in the dataset for completeness.

However, much needs to be done to prepare the data set for analysis. Lets start off by reading in the data set.

In [2]:
# Concatenating metro bike share data
bike_data = pd.DataFrame()

files_path = os.path.join('data', 'metro-bike-share-trips-****-**.csv')
files = glob.glob(files_path)

for file in files:
    csv = pd.read_csv(file)
    bike_data = pd.concat([bike_data, csv], sort=True)
    
bike_data.head(3)

,bike_id,bike_type,duration,end_lat,end_lon,end_station,end_station_id,end_time,passholder_type,plan_duration,start_lat,start_lon,start_station,start_station_id,start_time,trip_id,trip_route_category
0,12116,NaN,2,NaN,NaN,3000.0,NaN,2017-12-30 10:22:00,Monthly Pass,30.0,NaN,NaN,3000.0,NaN,2017-12-30 10:20:00,65463879,One Way
1,6370,NaN,1440,NaN,NaN,3000.0,NaN,2018-01-01 09:27:00,Walk-up,0.0,34.04607,-118.23309,3022.0,NaN,2017-12-29 08:03:00,65465211,One Way
2,6411,NaN,1440,NaN,NaN,3000.0,NaN,2018-01-01 18:24:00,Walk-up,0.0,34.05048,-118.25459,3007.0,NaN,2017-12-30 23:44:00,65522945,One Way


We should also pay close attention to the missingness, types, etc. of the data.

In [3]:
bike_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 639786 entries, 0 to 73866
Data columns (total 17 columns):
bike_id                639786 non-null object
bike_type              73867 non-null object
duration               639786 non-null int64
end_lat                630676 non-null float64
end_lon                630676 non-null float64
end_station            550561 non-null float64
end_station_id         89225 non-null float64
end_time               639786 non-null object
passholder_type        639786 non-null object
plan_duration          639402 non-null float64
start_lat              638432 non-null float64
start_lon              638432 non-null float64
start_station          550561 non-null float64
start_station_id       89225 non-null float64
start_time             639786 non-null object
trip_id                639786 non-null int64
trip_route_category    639786 non-null object
dtypes: float64(9), int64(2), object(6)
memory usage: 87.9+ MB


To no surprise, it is quite messy. Let's do some cleaning and prepare the data set for analysis.

In [4]:
# Converting time data to datetime format, takes some time..
bike_data['start_time'] = pd.to_datetime(bike_data['start_time'])
bike_data['end_time'] = pd.to_datetime(bike_data['end_time'])


# Combining latitudes and longitudes
bike_data['start_lat_lon'] = list(zip(bike_data['start_lat'], bike_data['start_lon']))
bike_data['end_lat_lon'] = list(zip(bike_data['end_lat'], bike_data['end_lon']))


# Capping duration at 24 hours,
# Metro Bike Share did not due so as they originally stated
bike_data['duration'][bike_data['duration'] > 1440] = 1440


# Converting all bike ids to strings
bike_data['bike_id'] = bike_data['bike_id'].astype(str)

# Keep and rearange columns of interest
cols_1 = 'bike_id bike_type passholder_type start_station end_station start_time'
cols_2 = ' end_time duration trip_route_category start_lat_lon end_lat_lon'
bike_cols =  (cols_1 + cols_2).split()
bike_data = bike_data[bike_cols].sort_values('start_time', ascending=False).reset_index(drop=True)

bike_data.head()

,bike_id,bike_type,passholder_type,start_station,end_station,start_time,end_time,duration,trip_route_category,start_lat_lon,end_lat_lon
0,12017,standard,Walk-up,3005.0,3082.0,2018-12-31 23:56:00,2019-01-01 00:22:00,26,One Way,"(34.0485, -118.25853700000002)","(34.04652, -118.23741100000001)"
1,12128,standard,Walk-up,3005.0,3082.0,2018-12-31 23:54:00,2019-01-01 00:22:00,28,One Way,"(34.0485, -118.25853700000002)","(34.04652, -118.23741100000001)"
2,12347,standard,Walk-up,4211.0,4210.0,2018-12-31 23:51:00,2019-01-01 01:28:00,97,One Way,"(33.984928000000004, -118.46996299999999)","(33.984341, -118.47155)"
3,5730,standard,Walk-up,3069.0,3069.0,2018-12-31 23:47:00,2018-12-31 23:54:00,7,Round Trip,"(34.05088, -118.248253)","(34.05088, -118.248253)"
4,6222,standard,Walk-up,3069.0,3069.0,2018-12-31 23:46:00,2018-12-31 23:55:00,9,Round Trip,"(34.05088, -118.248253)","(34.05088, -118.248253)"


Now that we have cleaned our data, we would also like to add a couple features that we feel will be useful for our analysis.

### Fare

We will calculate the estimated fare for every trip in the data set. We must note that Metro Bike Share lowered their prices in hopes of attracting more riders on July 12, 2018. 

Pricing _prior_ to the cuts:

>**Flex Pass (Year)**
>* Membership Cost: \$40
>* \$1.75 per trip per trip up to 30 minutes each
>* \$1.75 every extra hour
> 
>**Monthly Pass**
>* Membership Cost: \$20
>* Unlimited free trips up to 30 minutes
>* \$1.75 every extra hour
>
>***One Day Pass**
>* Membership Cost: \$7
>* Unlimited free trips up to 30 minutes
>* \$1.75 every extra hour
>
>**Walk-Up**
>* Membership Cost: \$0
>* \$3.50 per trip per trip up to 30 minutes each
>* \$3.50 every extra hour

Pricing _after_ cuts:

>**Annual Pass**
>* Membership Cost: \$150
>* Unlimited free trips up to 30 minutes
>* \$1.75 every extra hour
>
>**Monthly Pass**
>* Membership Cost: \$17
>* Unlimited free trips up to 30 minutes
>* \$1.75 every extra hour
>
>**One Day Pass**
>* Membership Cost: \$5
>* Unlimited free trips up to 30 minutes
>* \$1.75 every extra hour
>
>**Walk-Up**
>* Membership Cost: \$0
>* \$1.75 per trip per trip up to 30 minutes each
>* \$1.75 every extra hour
>
>***Flex Pass**
>* Membership Cost: \$40
>* Unlimited free trips up to 30 minutes
>* \$1.75 every extra hour

**(*) Estimated due to an incomplete record of pricing**

In [5]:
# Calculating fare for each trip prior to price cut
prior_cut = bike_data[bike_data['start_time'] < '2018-07-12']
prior_cut['estimated_fare'] = prior_cut.apply(lambda x: int(x['duration']/30.1)*1.75 
                                              if x['passholder_type'] != 'Walk-up' 
                                              else (int(x['duration']/30.1)+1)*1.75
                                              if x['passholder_type'] == 'Flex Pass'
                                              else ((int(x['duration']/30.1))*1.75)+7
                                              if x['passholder_type'] == 'One Day Pass'
                                              else (int(x['duration']/30.1)+1)*3.50, axis=1)

# Calculating fare for each trip after price cut
post_cut = bike_data[bike_data['start_time'] >= '2018-07-12']
post_cut['estimated_fare'] = post_cut.apply(lambda x: int(x['duration']/30.1)*1.75 
                                            if x['passholder_type'] != 'Walk-up'
                                            else ((int(x['duration']/30.1))*1.75)+5
                                            if x['passholder_type'] == 'One Day Pass'
                                            else (int(x['duration']/30.1)+1)*1.75, axis=1)

# Adding to bike data
fares = pd.concat([prior_cut['estimated_fare'], post_cut['estimated_fare']])
bike_data = pd.concat([bike_data, fares], ignore_index=False, axis=1)

bike_data.head()

,bike_id,bike_type,passholder_type,start_station,end_station,start_time,end_time,duration,trip_route_category,start_lat_lon,end_lat_lon,estimated_fare
0,12017,standard,Walk-up,3005.0,3082.0,2018-12-31 23:56:00,2019-01-01 00:22:00,26,One Way,"(34.0485, -118.25853700000002)","(34.04652, -118.23741100000001)",1.75
1,12128,standard,Walk-up,3005.0,3082.0,2018-12-31 23:54:00,2019-01-01 00:22:00,28,One Way,"(34.0485, -118.25853700000002)","(34.04652, -118.23741100000001)",1.75
2,12347,standard,Walk-up,4211.0,4210.0,2018-12-31 23:51:00,2019-01-01 01:28:00,97,One Way,"(33.984928000000004, -118.46996299999999)","(33.984341, -118.47155)",7.00
3,5730,standard,Walk-up,3069.0,3069.0,2018-12-31 23:47:00,2018-12-31 23:54:00,7,Round Trip,"(34.05088, -118.248253)","(34.05088, -118.248253)",1.75
4,6222,standard,Walk-up,3069.0,3069.0,2018-12-31 23:46:00,2018-12-31 23:55:00,9,Round Trip,"(34.05088, -118.248253)","(34.05088, -118.248253)",1.75


In [6]:
bike_data[bike_data['passholder_type']=='One Day Pass']

,bike_id,bike_type,passholder_type,start_station,end_station,start_time,end_time,duration,trip_route_category,start_lat_lon,end_lat_lon,estimated_fare
39,5718,standard,One Day Pass,4207.0,4207.0,2018-12-31 22:04:00,2019-01-01 00:24:00,140,Round Trip,"(34.000881, -118.46891000000001)","(34.000881, -118.46891000000001)",7.00
48,5992,standard,One Day Pass,3052.0,3078.0,2018-12-31 21:46:00,2018-12-31 22:10:00,24,One Way,"(34.051102, -118.26455700000001)","(34.064281, -118.238937)",0.00
55,5992,standard,One Day Pass,3031.0,3052.0,2018-12-31 21:28:00,2018-12-31 21:46:00,18,One Way,"(34.044701, -118.25244099999999)","(34.051102, -118.26455700000001)",0.00
62,6382,standard,One Day Pass,3067.0,3045.0,2018-12-31 21:05:00,2018-12-31 21:18:00,13,One Way,"(34.045181, -118.25023700000001)","(34.028511, -118.25666799999999)",0.00
65,12047,standard,One Day Pass,3006.0,3063.0,2018-12-31 21:02:00,2018-12-31 21:09:00,7,One Way,"(34.04554, -118.25666799999999)","(34.048038, -118.25373799999998)",0.00
66,6026,standard,One Day Pass,3006.0,3063.0,2018-12-31 21:01:00,2018-12-31 21:08:00,7,One Way,"(34.04554, -118.25666799999999)","(34.048038, -118.25373799999998)",0.00
84,6459,standard,One Day Pass,3067.0,3031.0,2018-12-31 19:50:00,2018-12-31 20:13:00,23,One Way,"(34.045181, -118.25023700000001)","(34.044701, -118.25244099999999)",0.00
87,6459,standard,One Day Pass,3062.0,3067.0,2018-12-31 19:35:00,2018-12-31 19:39:00,4,One Way,"(34.047748999999996, -118.243172)","(34.045181, -118.25023700000001)",0.00
93,6643,standard,One Day Pass,3047.0,3006.0,2018-12-31 19:01:00,2018-12-31 19:15:00,14,One Way,"(34.039982, -118.266403)","(34.04554, -118.25666799999999)",0.00
98,6343,standard,One Day Pass,3047.0,3006.0,2018-12-31 18:59:00,2018-12-31 19:15:00,16,One Way,"(34.039982, -118.266403)","(34.04554, -118.25666799999999)",0.00


### Weather

We will also add the weather conditions for each trip. We must first clean up the weather data so we can effectively merge with our bike data. 

Please note that these conditions are based on the relative conditions of Los Angeles at the time of the trip but may not completely reflect the conditions felt by the bike rider.

In [7]:
# Reading in weather data
weather_path = os.path.join('data', 'KLAX.txt')
weather_columns = 'Date Hour Temperature RH Windspeed Weather'.split()
weather = pd.read_csv(weather_path)[weather_columns]

# Converting date to datetime and getting dates of interest
weather['Date'] = pd.to_datetime(weather['Date'])
weather = weather[weather['Date'] >= '2016-07-07'].reset_index(drop=True)
weather['Hour'] = pd.to_timedelta(weather['Hour'], unit='hr')

# Converting weather codes to meanings
weather_dict = {'OVC': 'Overcast', 'BKN': 'Broken Clouds', 'CLR': 'Clear', 
                'HZ': 'Haze', 'FEW': 'Few Clouds', 'SCT': 'Scattered Clouds', 
                'BR': 'Mist', 'FG': 'Fog', '-RA': 'Light Rain', '-RA BR': 'Light Rain and Mist', 
                'RA BR': 'Rain and Mist', '+RA BR': 'Heavy Rain and Mist', '+RA': 'Heavy Rain', 'RA': 'Rain', 
                'BCFG': 'Patches of Fog', 'BCFG BR': 'Patches of Fog and Mist', 'FG BR': 'Fog and Mist', 
                'HZ FU': 'Haze and Smoke', 'MIFG': 'Shallow Fog', 'FU': 'Smoke', '-DZ': 'Light Drizzle',
                '-DZ BR': 'Light Drizzle and Mist', 'PRFG BR': 'Partial Fog and Mist', 'SM BR': 'Mist', 
                'PRFG HZ': 'Partial Fog and Mist', '-TSRA': 'Light Thunderstorm and Rain', 
                'BR -DZ': 'Mist and Light Drizzle', 'TSRA': 'Thunderstorm and Rain'}

weather['Weather'] = weather['Weather'].replace(weather_dict)
weather.head()

,Date,Hour,Temperature,RH,Windspeed,Weather
0,2016-07-07,00:00:00,65,81,5,Broken Clouds
1,2016-07-07,01:00:00,65,81,3,Overcast
2,2016-07-07,02:00:00,65,81,0,Overcast
3,2016-07-07,03:00:00,66,78,5,Overcast
4,2016-07-07,04:00:00,66,78,5,Overcast


Now we can combine it all together!

In [8]:
# Creating columns to merge on

bike_data['Hour'] = pd.to_timedelta(bike_data['start_time'].dt.hour, unit='hr')
bike_data['Date'] = pd.to_datetime(bike_data['start_time'].dt.date)

# Merging

bike_data = pd.merge(bike_data, weather, how='left', on=['Date', 'Hour'])
bike_data = bike_data.drop(['Hour', 'Date'], axis=1)

# Doing some final cleaning on the columns
bike_data.columns = [x.lower().replace('', '') for x in bike_data.columns]
bike_data.rename(columns={'rh': 'humidity'}, inplace=True)
order1 = 'bike_id bike_type passholder_type duration trip_route_category estimated_fare temperature humidity'
order2 = ' windspeed weather start_station end_station start_time end_time start_lat_lon end_lat_lon'
bike_data = bike_data[(order1+order2).split()]

bike_data.head()

,bike_id,bike_type,passholder_type,duration,trip_route_category,estimated_fare,temperature,humidity,windspeed,weather,start_station,end_station,start_time,end_time,start_lat_lon,end_lat_lon
0,12017,standard,Walk-up,26,One Way,1.75,54,17,5,Few Clouds,3005.0,3082.0,2018-12-31 23:56:00,2019-01-01 00:22:00,"(34.0485, -118.25853700000002)","(34.04652, -118.23741100000001)"
1,12128,standard,Walk-up,28,One Way,1.75,54,17,5,Few Clouds,3005.0,3082.0,2018-12-31 23:54:00,2019-01-01 00:22:00,"(34.0485, -118.25853700000002)","(34.04652, -118.23741100000001)"
2,12347,standard,Walk-up,97,One Way,7.00,54,17,5,Few Clouds,4211.0,4210.0,2018-12-31 23:51:00,2019-01-01 01:28:00,"(33.984928000000004, -118.46996299999999)","(33.984341, -118.47155)"
3,5730,standard,Walk-up,7,Round Trip,1.75,54,17,5,Few Clouds,3069.0,3069.0,2018-12-31 23:47:00,2018-12-31 23:54:00,"(34.05088, -118.248253)","(34.05088, -118.248253)"
4,6222,standard,Walk-up,9,Round Trip,1.75,54,17,5,Few Clouds,3069.0,3069.0,2018-12-31 23:46:00,2018-12-31 23:55:00,"(34.05088, -118.248253)","(34.05088, -118.248253)"


In [9]:
bike_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 639786 entries, 0 to 639785
Data columns (total 16 columns):
bike_id                639786 non-null object
bike_type              73867 non-null object
passholder_type        639786 non-null object
duration               639786 non-null int64
trip_route_category    639786 non-null object
estimated_fare         639786 non-null float64
temperature            639786 non-null int64
humidity               639786 non-null int64
windspeed              639786 non-null int64
weather                639682 non-null object
start_station          550561 non-null float64
end_station            550561 non-null float64
start_time             639786 non-null datetime64[ns]
end_time               639786 non-null datetime64[ns]
start_lat_lon          639786 non-null object
end_lat_lon            639786 non-null object
dtypes: datetime64[ns](2), float64(3), int64(4), object(7)
memory usage: 83.0+ MB


In [10]:
bike_data.to_csv('bike_clean.csv', index=False)